In [1]:
import torch
import os
from tqdm import tqdm
from helpers_function import *
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")
model = AutoModelForQuestionAnswering.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1", return_dict=False)

c:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at valhalla/longformer-base-4096-finetuned-squadv1 were not used when initializing LongformerForQuestionAnswering: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
- This IS expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
urls, company_names, segment = get_data()

In [3]:
questions = ['what is the name of this company?', 'when was this company established?', 'what is the company ISO certification?', 'what is the total number of employees?', 'what is the growth rate of this company?', 'what is the revenue of this company?', 'where is the global HQ location of this company?', 'on what field this company works?', 'what is the vision of this company?', 'what are the past awards that has been awarded to this company?', 'Describe about this company.', 'what is the name of the parent company?', 'what is the company external ICT spend?', 'what is the company R&D spend?', 'what is the company R&D focus?', 'what is the marketing budget of this company?', 'what is the number of patents of this company?']
print(len(questions))

17


In [4]:
os.mkdir(f"D:/Machine Learning/scrapper and extraction/valhalla_answers")

final_answer = []

for i, company in tqdm(enumerate(company_names)):
    answers = []
    for j in range(5):
        path = f"D:/Machine Learning/scrapper and extraction/raw_text/{company}/my_file_{j}.txt"
        text = get_text(path)

        if text == "":
            continue

        ans = []

        for ques in questions:
            text = text
            question = ques
            encoding = tokenizer(question, text, return_tensors="pt")
            input_ids = encoding["input_ids"]

            # default is local attention everywhere
            # the forward method will automatically set global attention on question tokens
            attention_mask = encoding["attention_mask"]

            start_scores, end_scores = model(input_ids, attention_mask=attention_mask)
            all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

            answer_tokens = all_tokens[torch.argmax(start_scores) :torch.argmax(end_scores)+1]
            answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

            ans.append(answer)
        
        answers.append(ans)

    final_answer.append(answers)

    # file = f"D:/Machine Learning/scrapper and extraction/valhalla_answers/{company}.txt"
    # with open(file, "w") as f:
    #     for k, ques in enumerate(questions):
    #         f.write(f'Questions: {ques}\nAnswer: {[ans[i] for ans in answers]}\n')

3it [29:32, 590.95s/it]


In [5]:
print(len(final_answer), len(final_answer[0]), len(final_answer[0][0]))

3 5 17


In [6]:
print(final_answer[0][0])

['Woebot for Adults', ' After two weeks', '<s>what is the company ISO certification?</s></s>Woebot for Adults is a non-prescription medical device under FDA enforcement discretion; it is not evaluated, cleared or approved by FDA', ' 2023', ' skyrocketing need for mental health care', ' skyrocketing need for mental health care', '<s>', ' mental health tool that answers the skyrocketing need for mental health care, and breaks down the systemic constraints that block equal access to it. Designed by humans, powered by AI, and grounded in science, Woebot easily integrates with health systems to provide evidenced-based behavioral health solutions that get people off a waitlist, and onto a path to feeling better.We didn’t just digitize CBT; we modeled the entire clinical journey to deliver tools and techniques with the insight and empathy inherent in human relationships. Woebot has been shown to establish a lasting working alliance with users akin to the bond formed between humans. It’s a pow

In [7]:
for i, company in enumerate(company_names):
    file = f"D:/Machine Learning/scrapper and extraction/valhalla_answers/{company}.txt"
    with open(file, "w") as f:
        for k, ques in enumerate(questions):
            f.write(f'Questions: {ques}\nAnswer: {[ans[k] for ans in final_answer[i]]}\n')

In [8]:
# for i, ques in enumerate(questions):
#     print(f'Questions: {ques}\nAnswer: {[ans[i] for ans in answers]}\n')